# Generate Bioclimatic Dataset

In this Notebook, the bioclimatic dataset is generated through the manipulation of species occurrence datasets from the Global Information Biodiversity Facility (GBIF) and the Portal da Biodiversidade do Instituto Chico Mendes de Conservação da Biodiversidade (ICMBio) and the join operation between these datasets and the one resultant from the interpolation proccess.

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def read_files(file_name):
    """
    Read the datasets with species occurrence data
    """
    
    if file_name == 'portal_bio':
        
        df = pd.read_excel('portalbio_export_16-08-2020-20-20-34.xlsx')
        
        # Select columns
        df = df[['Data do registro','Nome cientifico','Nome comum','Nivel taxonomico', 'Numero de individuos', 'Reino', 'Filo', 'Classe','Ordem', 'Familia', 'Genero', 'Especie', 'Localidade', 'Pais', 'Estado/Provincia','Municipio', 'Latitude', 'Longitude']]
        df = df.rename(columns={'Data do registro':'date', 'Latitude':'latitude', 'Longitude':'longitude', 'Especie':'species'})
        df = df[['date','latitude','longitude','species']]
        
        # Round coordinates
        df['latitude'] = df['latitude'].apply(lambda x: x/(10**(len(str(x))-2)))
        df['longitude'] = df['longitude'].apply(lambda x: x/(10**(len(str(x))-3)))
        
        # Filter unknown species
        df = df[df['species'] != 'Sem Informações'].reset_index(drop = True)
        
        return df
    
    elif file_name == 'GBIF':
        
        df = pd.read_excel('GBIF_occu.xlsx')
        
        # Select columns
        df = df[['eventDate','stateProvince','county','municipality','locality','decimalLatitude','decimalLongitude','kingdom','phylum','class','order','family','genus','species']]
        df = df.rename(columns={'eventDate':'date', 'decimalLatitude':'latitude', 'decimalLongitude':'longitude'})
        df = df[['date','latitude','longitude','species']]
        
        # Round coordinates
        df['latitude'] = df['latitude'].apply(lambda x: x/(10**(len(str(x))-4)))
        df['longitude'] = df['longitude'].apply(lambda x: x/(10**(len(str(x))-5)))
        
        # Filter unknown species        
        df = df.dropna(subset = ['species']).reset_index(drop = True)
        
        return df
    

def filter_files(df,ini_date, end_date, min_lat, min_long, max_lat, max_long):
    """
    Filter the datasets by date and coordinates (latitude and longitude)
    """
    
    # Filter by date
    df = df[df['date']>=ini_date]
    df = df[df['date']<=end_date]
    df = df.sort_values(by='date').reset_index(drop = True)
    
    # Filter by coordinates
    df = df[df['latitude'] >= min_lat]
    df = df[df['latitude'] <= max_lat]
    df = df[df['longitude'] >= min_long]
    df = df[df['longitude'] <= max_long]
    df = df.reset_index(drop = True)
    
    return df


def concatenate_dfs(df1, df2):
    """
    Concatenate the species occurrence datasets
    """

    df = pd.concat([df1,df2])
    df = df.sort_values(by='date').reset_index(drop = True)
    
    # Drop duplicates
    df = df.drop_duplicates(subset=['latitude', 'longitude', 'date', 'species'], keep='first').reset_index(drop=True)
    
    return df


def select_season(season, df):
    """
    Filter the dataset by season
    """
    
    if season == 'dry':
        
        # Filter by season
        mask1 = df['date'].map(lambda x: x.month) == 8
        mask2 = df['date'].map(lambda x: x.month) == 9
        mask3 = df['date'].map(lambda x: x.month) == 10
        df_1 = df[mask1]
        df_2 = df[mask2]
        df_3 = df[mask3]
        df_dry = pd.concat([df_1,df_2,df_3], ignore_index=True)
        
        return df_dry
        
    elif season == 'wet':
        
        # Filter by season
        mask1 = df['date'].map(lambda x: x.month) == 1
        mask2 = df['date'].map(lambda x: x.month) == 2
        mask3 = df['date'].map(lambda x: x.month) == 3
        df_1 = df[mask1]
        df_2 = df[mask2]
        df_3 = df[mask3]
        df_wet = pd.concat([df_1,df_2,df_3], ignore_index=True)
        
        return df_wet
    

def merge_datasets(season, df):
    """
    Merge the datasets in order to generate the bioclimatic dataset
    """
    
    # Read interpolated data
    df_interpolated = pd.read_excel('Interpolated_data_{}.xlsx'.format(season))
    
    # Format coordinates
    df_interpolated['latitude_'] = df_interpolated['latitude'].apply(lambda x: round(x,3))
    df_interpolated['longitude_'] = df_interpolated['longitude'].apply(lambda x: round(x,3))
    df['latitude_'] = df['latitude'].apply(lambda x: round(x,3))
    df['longitude_'] = df['longitude'].apply(lambda x: round(x,3))
    
    # Join
    df = df.merge(df_interpolated, on=['latitude_', 'longitude_']).reset_index(drop=True)
    
    # Count the numbers of occurrences
    occurrences = pd.DataFrame(df.groupby(by=['species'])['date'].count())
    occurrences = occurrences.rename(columns={'date':'qty'})
    
    # Filter minimum quantity
    occurrences = occurrences[occurrences['qty'] >= 17].reset_index()
    
    # Select columns
    list_columns = list(df.columns)
    list_columns.remove('latitude_x')
    list_columns.remove('longitude_x')
    list_columns.remove('latitude_')
    list_columns.remove('longitude_')
    df = df[list_columns]
    df = df.rename(columns={'latitude_y':'latitude','longitude_y':'longitude'})
    
    # Create new column
    df_interpolated.loc[:,'species'] = None

    # Add key column
    def key(lat,lon):
        return str(lat)+str(lon)

    df_interpolated['key'] = df_interpolated.apply(lambda x: key(x['latitude'],x['longitude']), axis=1)
    df['key'] = df.apply(lambda x: key(x['latitude'],x['longitude']), axis=1)

    # Filtering
    df_interpolated = df_interpolated.loc[~df_interpolated['key'].isin(df['key'].to_list())]
    df_interpolated = pd.concat([df_interpolated,df]).reset_index(drop=True)
    l_columns = occurrences['species'].to_list()+[None]
    df_interpolated = df_interpolated.loc[df_interpolated['species'].isin(l_columns)].reset_index(drop=True)
    
    # Encoding
    df_enc = pd.get_dummies(df_interpolated.species, prefix='species')

    # Add column
    df_bioclim = pd.merge(df_interpolated, df_enc, left_index=True, right_index=True)
    
    # Drop NaN
    list_columns.remove('latitude_y')
    list_columns.remove('longitude_y')
    list_columns.remove('date')
    list_columns.remove('species')
    df_bioclim = df_bioclim.dropna(subset=list_columns)
    
    # Drop columns
    df_bioclim.drop(labels=['latitude_','longitude_','species','key','date'], axis=1)
    df_bioclim = df_bioclim.reset_index(drop=True)
    
    df_bioclim.to_csv('Bioclimatic_dataset_{}.csv'.format(season), index=False)
    
    return df_bioclim

In [4]:
file_name_1 = 'portal_bio'
file_name_2 = 'GBIF'
df1 = read_files(file_name_1)
df2 = read_files(file_name_2)

ini_date = '2009-01-01'
end_date = '2019-01-01'
min_lat = -3.5
min_long = -60.8
max_lat = -2.8
max_long = -59.8

df1 = filter_files(df1,ini_date, end_date, min_lat, min_long, max_lat, max_long)
df2 = filter_files(df2,ini_date, end_date, min_lat, min_long, max_lat, max_long)

df = concatenate_dfs(df1, df2)

season = 'dry'

df = select_season(season, df)

df_bioclim = merge_datasets(season, df)
df_bioclim

,latitude,longitude,co(ppm),o3(ppb),nox(ppb),CPC3010,Acetonitrile(ppb),Isoprene(ppb),latitude_,longitude_,...,species_Thraupis episcopus,species_Thraupis palmarum,species_Todirostrum maculatum,species_Tringa solitaria,species_Troglodytes aedon,species_Turdus ignobilis,species_Turdus leucomelas,species_Tyrannulus elatus,species_Tyrannus melancholicus,species_Tyrannus savana
0,-3.5997,-60.429,0.135955,25.720111,1.550938,2079.278605,0.236997,2.766775,-3.600,-60.429,...,False,False,False,False,False,False,False,False,False,False
1,-3.5997,-60.428,0.135794,26.174311,1.548553,2097.671272,0.239230,2.770939,-3.600,-60.428,...,False,False,False,False,False,False,False,False,False,False
2,-3.5997,-60.427,0.135898,26.385876,1.551119,2099.094735,0.241002,2.769552,-3.600,-60.427,...,False,False,False,False,False,False,False,False,False,False
3,-3.5997,-60.426,0.136182,25.860619,1.559736,2108.608617,0.247720,2.711530,-3.600,-60.426,...,False,False,False,False,False,False,False,False,False,False
4,-3.5977,-60.430,0.136300,25.095534,1.552054,2102.775432,0.237204,2.851621,-3.598,-60.430,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168244,-3.1857,-60.131,0.146338,31.857591,5.313674,4614.214621,0.270687,1.298981,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
168245,-3.1857,-60.131,0.146338,31.857591,5.313674,4614.214621,0.270687,1.298981,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
168246,-3.1857,-60.131,0.146338,31.857591,5.313674,4614.214621,0.270687,1.298981,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
168247,-3.1857,-60.131,0.146338,31.857591,5.313674,4614.214621,0.270687,1.298981,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
